In [ ]:
Import

In [2]:
from tenpy.networks.mps import MPS, TransferMatrix
from tenpy.networks.site import Site,SpinSite,SpinHalfSite
from tenpy.models.spins import SpinModel
from tenpy.algorithms import dmrg
from tenpy.linalg.np_conserved import expm,Array
from tenpy.linalg.np_conserved import norm
from tenpy.models.lattice import Site, Chain
from tenpy.models.model import CouplingModel, NearestNeighborModel, MPOModel, CouplingMPOModel
from tenpy.linalg import np_conserved as npc
from tenpy.tools.params import get_parameter, unused_parameters
import tenpy.linalg.np_conserved as npc

import numpy as np
import matplotlib.pyplot as plt

import cmath
import math
import tenpy

from scipy.sparse.linalg import eigs, eigsh
from scipy.sparse import diags, kron, csr_matrix,coo_matrix, bmat
from scipy.linalg import expm, sinm, cosm


Heisenberg Chain, finite, with open, periodic or antiperiodic boundary conditions

In [2]:
from tenpy.models.model import CouplingMPOModel
class HeisenbergChain_boundary_conditions(CouplingMPOModel):
    default_lattice = "Chain"
    force_default_lattice = True

    def init_sites(self, model_params):
        return SpinSite(S=1,conserve=None) 
        
    def init_terms(self, model_params):
        site=SpinSite(S=1,conserve=None)
        J_z = 1.  # the strength
        J_x = 1.
        J_y = 1. 
        D=model_params["D"]
        bc=model_params["bc"]
        for u1, u2, dx in self.lat.pairs['nearest_neighbors']:
            #print(u1,u2,dx)
            self.add_coupling(J_z, u1, 'Sz', u2, 'Sz', dx)
            self.add_coupling(J_x, u1, 'Sx', u2, 'Sx', dx)
            self.add_coupling(J_y, u1, 'Sy', u2, 'Sy', dx)
            self.add_onsite(D, u1, 'Sz Sz')
        L=model_params["L"]-1
        self.add_coupling_term(J_z, 0, L, 'Sz', 'Sz', op_string='Id')
        self.add_coupling_term(bc, 0, L, 'Sx', 'Sx', op_string='Id')
        self.add_coupling_term(bc, 0, L, 'Sy', 'Sy', op_string='Id')

Time reversal order parameter for infinite MPS 

In [ ]:
def DMRG_psi(N,D):
    model = SpinModel({"S":1,"L": 10, "Jx": 1.,"Jy":1.,"muJ":0,"Jz": 1.,"D":D,"hx": 0, "conserve":None,"bc_MPS": "infinite"})
    sites = model.lat.mps_sites()
    psi = MPS.from_product_state(sites, ["up", "down"] * (N//2), "infinite")
    dmrg_params = {"trunc_params": {"chi_max": 100, "svd_min": 1.e-7}, "mixer": True}
    info = dmrg.run(psi, model, dmrg_params)
    return psi

def norm_U(U):
    return np.sqrt((np.trace(np.dot(U,U.conj().T))*U.shape[0]))

def rotation(alpha,operator_tilde):
        return expm(1j*cmath.pi*(operator_tilde)*alpha)
    
def time_reversal_projective_representations_eigvals(psi):
    L=psi.L
    row  = np.array([2,1,1,0])
    col  = np.array([1,2,0,1])
    data = np.array([1,-1,1,-1])
    S_y=1/np.sqrt(2)*1j*coo_matrix((data, (row, col)), shape=(3, 3))
    R_TR = npc.Array.from_ndarray_trivial(rotation(1,S_y.toarray()),labels=['p','p*'])
    psi2=psi.copy()
    for i in range(L):
        psi2.apply_local_op(i,R_TR)
    
    eigvals1, vawr = TransferMatrix(psi,psi2,shift_bra=0,shift_ket=0).eigenvectors(1)
    #print(eigvals1)
    TR=vawr[0].split_legs().to_ndarray()
    TR=norm_U(TR)*TR
    return TR,eigvals1[0]

    
def TR_order_parameter(TR,eigval1):
    eigval1=math.isclose(eigval1, 1, rel_tol=1e-02, abs_tol=0.0)
    if (eigval1):
        return 1/(min(TR.shape))*np.trace(np.dot(TR,TR.conj())).real   
    else: 
        return 0

#Just execute this with N= length, D= sing
def O_TR_D(N,D):
    psi=DMRG_psi(N,D)
    TR, eigval1=time_reversal_projective_representations_eigvals(psi)
    print(eigval1)
    return TR_order_parameter(TR,eigval1)

Z2xZ2 order parameter for infinite MPS

In [ ]:
def DMRG_psi(N,D):
    model = SpinModel({"S":1,"L": 10, "Jx": 1.,"Jy":1.,"muJ":0,"Jz": 1.,"D":D,"hx": 0, "conserve":None,"bc_MPS": "infinite"})
    sites = model.lat.mps_sites()
    psi = MPS.from_product_state(sites, ["up", "down"] * (N//2), "infinite")
    dmrg_params = {"trunc_params": {"chi_max": 100, "svd_min": 1.e-7}, "mixer": True}
    info = dmrg.run(psi, model, dmrg_params)
    return psi

def norm_U(U):
    return np.sqrt((np.trace(np.dot(U,U.conj().T))*U.shape[0]))

def rotation(alpha,operator_tilde):
    return expm(1j*cmath.pi*(operator_tilde)*alpha)
    
def Z2xZ2_projective_representations_eigvals(psi):
    L=psi.L
    row  = np.array([0,1,2])
    col  = np.array([0,1,2])
    data = np.array([1,0,-1])
    S_z=coo_matrix((data, (row, col)), shape=(3, 3))

    row  = np.array([2,1,1,0])
    col  = np.array([1,2,0,1])
    data = np.array([1,1,1,1])
    S_x=np.sqrt(1/2)*coo_matrix((data, (row, col)), shape=(3, 3))
    
    row  = np.array([2,1,1,0])
    col  = np.array([1,2,0,1])
    data = np.array([1,1,-1,-1])
    S_y=np.sqrt(1/2*1j)*coo_matrix((data, (row, col)), shape=(3, 3))
    
    row  = np.array([2,0,1])
    col  = np.array([0,2,1])
    data = np.array([1,1,-1])
    S_x_S_z_rotation=coo_matrix((data, (row, col)), shape=(3, 3))


    R_z = npc.Array.from_ndarray_trivial(rotation(1,S_z.toarray()),labels=['p','p*'])
    R_x = npc.Array.from_ndarray_trivial(rotation(1,S_x.toarray()),labels=['p','p*'])
    
    psi2=psi.copy()
    for i in range(L):
        psi2.apply_local_op(i,R_z)
    psi3=psi.copy()
    for i in range(L):
        psi3.apply_local_op(i,R_x)
    
    eigvals1, vawr = TransferMatrix(psi,psi2,shift_bra=0,shift_ket=0).eigenvectors(1)
    
    Uz=vawr[0].split_legs().to_ndarray()
    eigvals2, vawr= TransferMatrix(psi,psi3,shift_bra=0,shift_ket=0).eigenvectors(1)
    Ux=vawr[0].split_legs().to_ndarray()
    Ux=norm_U(Ux)*Ux
    Uz=norm_U(Uz)*Uz
    return Uz,Ux,eigvals1.real[0],eigvals2.real[0]




def Z2xZ2_order_parameter(Uz,Ux,eigval1,eigval2):
    eigval1=math.isclose(eigval1, 1, rel_tol=1e-02, abs_tol=0.0)
    eigval2=math.isclose(eigval2, 1, rel_tol=1e-02, abs_tol=0.0)
    if (eigval1 and eigval2):
        return 1/(min(Ux.shape))*np.trace(np.dot(np.dot(Uz,Ux),np.dot(Uz.conj(),Ux.conj()))).real
        
    else: 
        return 0
    



#Just execute this with N= length, D= single ion anisotropy
def O_Z2_Z2_D(N,D):
    psi=DMRG_psi(N,D)
    Uz,Ux,eigval1,eigval2=Z2xZ2_projective_representations_eigvals(psi)
    print(eigval1,eigval2)
    return Z2xZ2_order_parameter(Uz,Ux,eigval1,eigval2)


Haldane and surface gap

In [ ]:
gap_haldane_even=[]
gap_surface_even=[]
x_data=[]
for N in np.arange(10,161,30):
    energies=[]
    N = N # number of sites
    model = SpinModel({"S":1,"L": N, "Jx": 1.,"Jy":1.,"muJ":0,"Jz": 1.,"D":0,"hx": 0, "conserve":'Sz',"bc_MPS": "finite"})
    sites = model.lat.mps_sites()
    psi0 = MPS.from_product_state(sites, ["up", "down"] * (N//2), "finite")
    dmrg_params = {"trunc_params": {"chi_max": 200, "svd_min": 1.e-7}, "mixer": True}
    states = dmrg_params['orthogonal_to'] = []
    for i in range(3):
        psi = psi0.copy()
        results = dmrg.run(psi, model, dmrg_params)
        energies.append(results['E'])
        states.append(psi)
    gap_surface_even.append(energies[1]-energies[0])
    gap_haldane_even.append(energies[2]-energies[0])

    x_data.append(N)
    print(N,end="\r")

Correlation length via transfermatrix

In [ ]:
D_list=[]
c_list=[]
for D in np.arange(0,1.5,0.1):
    N = 2
    model = SpinModel({"S":1,"L": N, "Jx": 1.,"Jy":1.,"Jz": 1.,"D":D,"g": 0, "conserve":'best',"bc_MPS": "infinite"})
    sites = model.lat.mps_sites()
    psi = MPS.from_product_state(sites, ["up", "down"] * (N//2), "infinite")
    dmrg_params = {"trunc_params": {"chi_max": 50, "svd_min": 1.e-7}, "mixer": True}
    info = dmrg.run(psi, model, dmrg_params)
    c_list.append(psi.correlation_length())
    D_list.append(D)

Convergence of entanglement entropy, Area Law

In [3]:
x_data=[]
EE=[]
for N in np.arange(10,45,4):
    #N = 9 # number of sites
    #model = SpinModel({'S':1,'L': N,
     #              'Jx': 1., 'Jy': 1., 'Jz': 1.,
      #             'bc_MPS': 'finite', 'bc_x': 'periodic',
       #            'order': 'folded','conserve': None})
    model = SpinModel({"S":1,"L": N, "Jx": 1.,"Jy":1.,"Jz": 1.,"D":0,"g": 0, "conserve":None,"bc_MPS": "finite"})
    sites = model.lat.mps_sites()
    psi = MPS.from_product_state(sites, (["up", "down"] * (N//2)), "finite")
    dmrg_params = {"trunc_params": {"chi_max": 100, "svd_min": 1.e-7}, "mixer": True}
    info = dmrg.run(psi, model, dmrg_params)
    EE.append(psi.entanglement_entropy(n=1, bonds=[int(N/2)], for_matrix_S=False))
    x_data.append(N)



Reading 'bc_MPS'='finite' for config SpinModel
Reading 'S'=1 for config SpinModel
Reading 'conserve'=None for config SpinModel
Reading 'L'=10 for config SpinModel
Reading 'Jx'=1.0 for config SpinModel
Reading 'Jy'=1.0 for config SpinModel
Reading 'Jz'=1.0 for config SpinModel
Reading 'D'=0 for config SpinModel
Subconfig 'trunc_params'=Config(<3 options>, 'trunc_params') for config DMRG
Reading 'mixer'=True for config DMRG


/home/rschulz/anaconda3/lib/python3.8/site-packages/tenpy/tools/params.py:225: UserWarning: unused option ['g'] for config SpinModel

  warnings.warn(msg.format(keys=sorted(unused), name=self.name))


sweep 10, age = 10
Energy = -12.8945601322043135, S = nan, norm_err = 2.1e-11
Current memory usage 136.6 MB, time elapsed: 10.8 s
Delta E = nan, Delta S = 0.0000e+00 (per sweep)
max_trunc_err = 1.1350e-12, max_E_trunc = 7.3115e-12
MPS bond dimensions: [3, 9, 27, 81, 100, 81, 27, 9, 3]
disable mixer after 15 sweeps, final amplitude 3.05e-10
sweep 20, age = 10
Energy = -12.8945601322042585, S = 1.1684060642655454, norm_err = 9.6e-12
Current memory usage 136.6 MB, time elapsed: 17.9 s
Delta E = 5.5067e-15, Delta S = nan (per sweep)
max_trunc_err = 5.6586e-13, max_E_trunc = 6.7200e-12
MPS bond dimensions: [3, 9, 27, 81, 100, 81, 27, 9, 3]
sweep 30, age = 10
Energy = -12.8945601322042602, S = 1.1684060642708161, norm_err = 2.3e-11
Current memory usage 136.6 MB, time elapsed: 23.4 s
Delta E = -1.7764e-16, Delta S = 5.2707e-13 (per sweep)
max_trunc_err = 5.6752e-13, max_E_trunc = 6.7271e-12
MPS bond dimensions: [3, 9, 27, 81, 100, 81, 27, 9, 3]
DMRG finished after 30 sweeps.
total size = 10, 

/home/rschulz/anaconda3/lib/python3.8/site-packages/tenpy/tools/params.py:225: UserWarning: unused option ['g'] for config SpinModel

  warnings.warn(msg.format(keys=sorted(unused), name=self.name))


sweep 10, age = 14
Energy = -18.4598526614705918, S = nan, norm_err = 1.4e-09
Current memory usage 151.9 MB, time elapsed: 29.3 s
Delta E = nan, Delta S = 0.0000e+00 (per sweep)
max_trunc_err = 6.9500e-11, max_E_trunc = 3.3289e-10
MPS bond dimensions: [3, 9, 27, 81, 100, 100, 100, 100, 100, 81, 27, 9, 3]
disable mixer after 15 sweeps, final amplitude 3.05e-10
sweep 20, age = 14
Energy = -18.4598526614735619, S = 1.2506057422988013, norm_err = 1.1e-09
Current memory usage 151.9 MB, time elapsed: 49.0 s
Delta E = -2.9701e-13, Delta S = nan (per sweep)
max_trunc_err = 3.4150e-11, max_E_trunc = 3.2128e-10
MPS bond dimensions: [3, 9, 27, 81, 100, 100, 100, 100, 100, 81, 27, 9, 3]
sweep 30, age = 14
Energy = -18.4598526614680871, S = 1.2506057427095967, norm_err = 1.1e-09
Current memory usage 151.9 MB, time elapsed: 64.5 s
Delta E = 5.4747e-13, Delta S = 4.1080e-11 (per sweep)
max_trunc_err = 3.4686e-11, max_E_trunc = 3.2704e-10
MPS bond dimensions: [3, 9, 27, 81, 100, 100, 100, 100, 100, 81

...

Subconfig 'trunc_params'=Config(<3 options>, 'trunc_params') for config DMRG
Reading 'mixer'=True for config DMRG
sweep 10, age = 34
Energy = -46.4440738761497300, S = nan, norm_err = 1.9e-07
Current memory usage 169.5 MB, time elapsed: 149.5 s
Delta E = nan, Delta S = 0.0000e+00 (per sweep)
max_trunc_err = 3.7785e-09, max_E_trunc = 1.4730e-08
MPS bond dimensions: [3, 9, 27, 81, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 81, 27, 9, 3]
disable mixer after 15 sweeps, final amplitude 3.05e-10
sweep 20, age = 34
Energy = -46.4440738776998927, S = 1.4336911134252333, norm_err = 6.5e-08
Current memory usage 169.5 MB, time elapsed: 217.4 s
Delta E = -1.5502e-10, Delta S = nan (per sweep)
max_trunc_err = 1.8728e-09, max_E_trunc = 1.4552e-08
MPS bond dimensions: [3, 9, 27, 81, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 81, 27, 9, 3]


/home/rschulz/anaconda3/lib/python3.8/site-packages/tenpy/tools/params.py:225: UserWarning: unused option ['g'] for config SpinModel

  warnings.warn(msg.format(keys=sorted(unused), name=self.name))


Subconfig 'trunc_params'=Config(<3 options>, 'trunc_params') for config DMRG
Reading 'mixer'=True for config DMRG
sweep 10, age = 38
Energy = -52.0490765439815277, S = nan, norm_err = 3.9e-07
Current memory usage 172.7 MB, time elapsed: 183.8 s
Delta E = nan, Delta S = 0.0000e+00 (per sweep)
max_trunc_err = 4.3233e-09, max_E_trunc = 1.6985e-08
MPS bond dimensions: [3, 9, 27, 81, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 81, 27, 9, 3]
disable mixer after 15 sweeps, final amplitude 3.05e-10
sweep 20, age = 38
Energy = -52.0490765457586022, S = 1.4471662112584802, norm_err = 1.1e-07
Current memory usage 172.7 MB, time elapsed: 268.4 s
Delta E = -1.7771e-10, Delta S = nan (per sweep)
max_trunc_err = 2.1455e-09, max_E_trunc = 1.6840e-08
MPS bond dimensions: [3, 9, 27, 81, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100,

/home/rschulz/anaconda3/lib/python3.8/site-packages/tenpy/tools/params.py:225: UserWarning: unused option ['g'] for config SpinModel

  warnings.warn(msg.format(keys=sorted(unused), name=self.name))


Subconfig 'trunc_params'=Config(<3 options>, 'trunc_params') for config DMRG
Reading 'mixer'=True for config DMRG
sweep 10, age = 42
Energy = -57.6545285550589952, S = nan, norm_err = 1.7e-06
Current memory usage 177.1 MB, time elapsed: 216.2 s
Delta E = nan, Delta S = 0.0000e+00 (per sweep)
max_trunc_err = 4.6693e-09, max_E_trunc = 1.8240e-08
MPS bond dimensions: [3, 9, 27, 81, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 81, 27, 9, 3]
disable mixer after 15 sweeps, final amplitude 3.05e-10
sweep 20, age = 42
Energy = -57.6545285560124654, S = 1.4576735292371745, norm_err = 3.4e-07
Current memory usage 177.1 MB, time elapsed: 305.8 s
Delta E = -9.5347e-11, Delta S = nan (per sweep)
max_trunc_err = 2.3191e-09, max_E_trunc = 1.8147e-08
MPS bond dimensions: [3, 9, 27, 81, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100,

Correlation functions

In [ ]:
N=80
model = SpinModel({"S":1,"L": N, "Jx": 1.,"Jy":1.,"Jz": 1.,"D":0 ,"g": 0, "conserve":None,"bc_MPS": "finite"})
sites = model.lat.mps_sites()
psi = MPS.from_product_state(sites, ["up", "down"] * (N//2), "finite")
dmrg_params = {"trunc_params": {"chi_max": 250, "svd_min": 1.e-7}, "mixer": True, "max_sweeps": 50}
info = dmrg.run(psi, model, dmrg_params)


In [ ]:
X = psi2.expectation_value("Sx")
Z = psi2.expectation_value("Sz")
x = np.arange(psi2.L)
plt.figure()
plt.plot(x, Z, label="Z")
plt.plot(x, X, label="X")   # note: it's clear that this is zero due to charge conservation!
plt.xlabel("Site i",fontsize=14)
plt.ylabel(r"$S_z^i , S_x^i$",fontsize=14)
plt.yticks(fontsize=14)
plt.xticks(fontsize=14)
plt.legend()
plt.show()


i0 = psi2.L // 4  # for fixed `i`
j = np.arange(i0 + 1, psi2.L)
XX = psi2.term_correlation_function_right([("Sx", 0)], [("Sx", 0)], i_L=i0, j_R=j)
XX_disc = XX - X[i0] * X[j]
ZZ = psi2.term_correlation_function_right([("Sz", 0)], [("Sz", 0)], i_L=i0, j_R=j)
ZZ_disc = ZZ - Z[i0] * Z[j]

dx = j - i0
plt.figure()
plt.plot(dx, XX_disc, label="X X")
plt.plot(dx, ZZ_disc, label="Z Z")
plt.yticks(fontsize=14)
plt.xticks(fontsize=14)
plt.xlabel(r"distance site i,j",fontsize=14)
plt.ylabel(r" $\langle A_i A_j\rangle - \langle A_i \rangle\langle A_j\rangle$",fontsize=14)
plt.legend()
#plt.loglog()
plt.show()

String order parameter

In [ ]:
def DMRG_psi(N,D):
    model = SpinModel({"S":1,"L": N, "Jx": 1.,"Jy":1.,"muJ":0,"Jz": 1.,"D":D,"hx": 0, "conserve":None,"bc_MPS": "infinite"})
    sites = model.lat.mps_sites()
    psi = MPS.from_product_state(sites, ["up", "down"] * (N//2), "infinite")
    dmrg_params = {"trunc_params": {"chi_max": 200, "svd_min": 1.e-7}, "mixer": True,'max_sweeps':50}
    info = dmrg.run(psi, model, dmrg_params)
    return psi

x_data=[]
SO_x=[]
SO_z=[]
def string_order():
    #L=62
    for D in np.arange(0.99,1.1,0.03):
        L=80
        print(L, end='\r')
        x_data.append(L)
        
        psi=DMRG_psi(L,D)
        Sz = psi.sites[0].Sz
        Sz_exp=npc.expm(1.j * np.pi * Sz)
        so_z=(psi.correlation_function("Sz", "Sz",opstr=Sz_exp,str_on_first=False))

        Sx = psi.sites[0].Sx
        Sx_exp=npc.expm(1.j * np.pi * Sx)
        so_x=(psi.correlation_function("Sx", "Sx",opstr=Sx_exp,str_on_first=False))

        SO_z.append(so_z[0][-1])
        SO_x.append(so_x[0][-1])

    print("SO_z",SO_z)  
    print("SO_x",SO_z) 
    print(x_data)
    plt.grid()
    plt.ylabel("SOP",fontsize=14)
    plt.xlabel("D",fontsize=14)
    plt.yticks(fontsize=14)
    plt.xticks(fontsize=14)
    plt.plot(x_data,SO_x,"d",label=r'$SOP(S_x)$')
    plt.plot(x_data,SO_z,"^",label=r'$SOP(S_z)$')
    plt.legend()

string_order()